In [2]:
from App import App
import cv2
import numpy as np

app = App()

Setting YOLO model for detect ...
Set reid with histogram
Setting the dictionary {name: vector feature} from load_data_base() ...
Set face recognition model with face_recognition 
Setup done!


In [4]:
image1 = cv2.imread('data_test/per1_1.jpg')
# object_images, results = app.detect_objects(image1)
results = app.yolo_model(image1)

0: 640x480 1 person, 533.6ms
Speed: 40.6ms preprocess, 533.6ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 480)


In [6]:
boxes = results[0].boxes

In [11]:
print(boxes[0].xywh[0][1].item())

541.8342895507812


In [ ]:
import cv2
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms

class SiameseNetwork(nn.Module):
    def __init__(self, base_model):
        super(SiameseNetwork, self).__init__()
        self.base_model = base_model

    def forward(self, x1, x2):
        # Forward pass for both images
        out1 = self.base_model(x1)
        out2 = self.base_model(x2)
        return out1, out2

# # Create a base model (ResNet-18 in this case)
# base_model = models.resnet18(pretrained=True)
# # Modify the final fully connected layer to output a suitable dimension
# base_model.fc = nn.Linear(base_model.fc.in_features, 256)

# # Create a Siamese Network
# siamese_net = SiameseNetwork(base_model)

# Create a base model (ResNet-18 in this case)
base_model = models.resnet18(pretrained=True)
base_model.fc = nn.Linear(base_model.fc.in_features, 256)

# Quantize the model
quantized_model = torch.quantization.quantize_dynamic(
    base_model, {nn.Linear}, dtype=torch.qint8
)

# Create a Siamese Network with the quantized model
siamese_net = SiameseNetwork(quantized_model)

# Example forward pass with two images
# image1 = torch.randn(1, 3, 224, 224)  # replace with your image dimensions
# image2 = torch.randn(1, 3, 224, 224)  # replace with your image dimensions

# Example usage with OpenCV images
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

# Read images using OpenCV
image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

# Check if images are read successfully
if image1 is None or image2 is None:
    print("Error: One or both images could not be read.")
else:
    # Convert OpenCV images to PyTorch tensors without PIL Image
    def preprocess_image(image):
        # Convert BGR to RGB
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        # Normalize to the range [0, 1]
        image_tensor = torch.from_numpy(image_rgb / 255.0).permute(2, 0, 1).float()
        # Add batch dimension
        image_tensor = image_tensor.unsqueeze(0)
        return image_tensor

    # Preprocess images
    input_tensor1 = preprocess_image(image1)
    input_tensor2 = preprocess_image(image2)

    # Example forward pass with two images
    embedding1, embedding2 = siamese_net(input_tensor1, input_tensor2)

    # Compute the similarity score (cosine similarity in this case)
    similarity_score = nn.functional.cosine_similarity(embedding1, embedding2)
    print(f"Similarity Score: {similarity_score.item()}")
    # Resnet50 đang là 43.6s
    # Resnet18 đang là 17.5s
    # Sử dụng thêm Quantom đang là 11.0s




In [ ]:
import cv2
import numpy as np

def color_histogram_similarity(image1, image2):
    hist1 = cv2.calcHist([image1], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    hist2 = cv2.calcHist([image2], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])

    # Normalize histograms
    hist1 = cv2.normalize(hist1, hist1).flatten()
    hist2 = cv2.normalize(hist2, hist2).flatten()

    # Clip values to ensure they are within valid range
    hist1 = np.clip(hist1, 0, 1)
    hist2 = np.clip(hist2, 0, 1)
    import matplotlib.pyplot as plt
    # Display histograms
    plt.plot(hist1, color='red', label='Image 1')
    plt.plot(hist2, color='blue', label='Image 2')
    plt.legend()
    plt.show()
    # Calculate histogram intersection similarity
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_INTERSECT)
    return similarity


# Example usage
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

image1 = cv2.imread(image_path1)
image2 = cv2.imread(image_path2)

# Resize images for consistency (optional)
image1 = cv2.resize(image1, (224, 224))
image2 = cv2.resize(image2, (224, 224))

# Calculate color histogram similarity
similarity_score = color_histogram_similarity(image1, image2)
print(f"Color Histogram Similarity Score: {similarity_score}")


In [11]:
import cv2
import numpy as np

def are_images_equal(image1, image2):
    # Đọc ảnh
    img1 = cv2.imread(image1)
    img2 = cv2.imread(image2)

    # Chuyển đổi ảnh sang không gian màu HSV
    hsv1 = cv2.cvtColor(img1, cv2.COLOR_BGR2HSV)
    hsv2 = cv2.cvtColor(img2, cv2.COLOR_BGR2HSV)

    # Tính histogram của ảnh
    hist1 = cv2.calcHist([hsv1], [0, 1], None, [180, 256], [0, 180, 0, 256])
    hist2 = cv2.calcHist([hsv2], [0, 1], None, [180, 256], [0, 180, 0, 256])

    # Chuẩn hóa histogram
    cv2.normalize(hist1, hist1, 0, 1, cv2.NORM_MINMAX)
    cv2.normalize(hist2, hist2, 0, 1, cv2.NORM_MINMAX)

    # Tính độ tương đồng giữa hai histogram
    similarity = cv2.compareHist(hist1, hist2, cv2.HISTCMP_CORREL)

    # Xác định ngưỡng để quyết định hai ảnh có chụp cùng đối tượng hay không
    threshold = 0.8  # Ngưỡng có thể điều chỉnh

    if similarity > threshold:
        return True
    else:
        return False

# Đường dẫn của hai ảnh cần so sánh
image_path1 = "data_test/test_reid/tower0_0.jpg"
image_path2 = "data_test/test_reid/tower1_0.jpg"

# Kiểm tra xem hai ảnh có chụp cùng đối tượng hay không
result = are_images_equal(image_path1, image_path2)

if result:
    print("Hai ảnh chụp cùng đối tượng.")
else:
    print("Hai ảnh không chụp cùng đối tượng.")


Hai ảnh không chụp cùng đối tượng.


In [2]:
cam1 = '/home/harito/Videos/Cam1.mp4'
cam2 = '/home/harito/Videos/Cam2.mp4'

cap1 = cv2.VideoCapture(cam1)
cap2 = cv2.VideoCapture(cam2)

In [3]:
ret1, frame1 = cap1.read()
ret2, frame2 = cap2.read()
real_frame = cv2.hconcat([frame1, frame2])

frame1_detect = app.detect_objects(frame1)
frame2_detect = app.detect_objects(frame2)
frame1_detect = cv2.cvtColor(np.array(app.draw_picture_detect(frame1_detect[1])), cv2.COLOR_RGB2BGR)
frame2_detect = cv2.cvtColor(np.array(app.draw_picture_detect(frame2_detect[1])), cv2.COLOR_RGB2BGR)

detect_frame = cv2.hconcat([frame1_detect, frame2_detect])


0: 384x640 2 persons, 1 couch, 1 dining table, 1 tv, 333.2ms
Speed: 27.0ms preprocess, 333.2ms inference, 3.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1 tv, 631.7ms
Speed: 2.8ms preprocess, 631.7ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)


In [4]:
print(real_frame.shape)
print(detect_frame.shape)

(720, 2560, 3)
(720, 2560, 3)


In [5]:
frame1_detect = app.detect_objects(frame1)


0: 384x640 2 persons, 1 couch, 1 dining table, 1 tv, 417.1ms
Speed: 23.3ms preprocess, 417.1ms inference, 2.5ms postprocess per image at shape (1, 3, 384, 640)


In [6]:
print(frame1_detect[1])

[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant'

In [13]:
detected_objects = []
# for box in frame1_detect[1][0].boxes:
#     detected_objects.append(box.cls())
# print(detected_objects)
print(frame1_detect[1][0].boxes)

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([ 0., 62., 57., 60.,  0.])
conf: tensor([0.9410, 0.8969, 0.5564, 0.4381, 0.3334])
data: tensor([[1.7601e+01, 1.6149e+00, 3.4273e+02, 7.1265e+02, 9.4102e-01, 0.0000e+00],
        [5.9464e+02, 7.3777e+01, 8.3778e+02, 2.8694e+02, 8.9687e-01, 6.2000e+01],
        [0.0000e+00, 3.4843e+02, 4.9174e+02, 6.8333e+02, 5.5641e-01, 5.7000e+01],
        [4.0832e+02, 5.0881e+02, 7.7487e+02, 7.1443e+02, 4.3811e-01, 6.0000e+01],
        [1.0111e+03, 1.5856e+00, 1.2793e+03, 7.1145e+02, 3.3337e-01, 0.0000e+00]])
id: None
is_track: False
orig_shape: (720, 1280)
shape: torch.Size([5, 6])
xywh: tensor([[ 180.1632,  357.1332,  325.1244,  711.0366],
        [ 716.2089,  180.3609,  243.1406,  213.1676],
        [ 245.8676,  515.8783,  491.7352,  334.8988],
        [ 591.5944,  611.6189,  366.5477,  205.6145],
        [1145.1799,  356.5157,  268.2527,  709.8602]])
xywhn: tensor([[0.1408, 0.4960, 0.2540, 0.9876],
        [0.5595, 0.2505, 0.190

In [30]:
# print(frame1_detect[1][0].boxes[0])
type(frame1_detect[1][0].boxes[0].cls.item())

float

In [2]:
import pickle

face_db_regis = pickle.loads(open("database/face_dictionary.pkl", "rb").read())

In [4]:
print(type(face_db_regis))
print(type(face_db_regis['encodings']))
print(face_db_regis['encodings'])

<class 'dict'>
<class 'list'>
[array([-0.06562555,  0.05791132,  0.05755185, -0.01877296, -0.04392779,
       -0.04681808, -0.01198823, -0.1409594 ,  0.14729714, -0.07548039,
        0.22148336, -0.06590459, -0.16843571, -0.10562707, -0.03778323,
        0.06519379, -0.18211022, -0.07301479, -0.06179518, -0.00868598,
        0.05805406,  0.02230086,  0.06453081,  0.03300014, -0.03951874,
       -0.34023571, -0.10079665, -0.144089  ,  0.00364012, -0.03615012,
       -0.02425949,  0.03060787, -0.17885473, -0.05996694, -0.01931695,
        0.01270618, -0.0592573 ,  0.06139456,  0.192431  , -0.01054506,
       -0.20676351, -0.01790451, -0.00918657,  0.25038904,  0.19814208,
        0.05069359, -0.04618467, -0.06285091,  0.10883839, -0.14116317,
        0.01082987,  0.1791468 ,  0.13339551,  0.04796259,  0.01227199,
       -0.08485886, -0.01884955,  0.04639325, -0.13455169,  0.04076411,
        0.07082561, -0.12931941, -0.0410993 , -0.04019939,  0.18569112,
        0.03175884, -0.11753236, 